In [ ]:
import os
import sys
from os import path
import numpy as np
import pandas as pd
from scipy import sparse, io
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Loading Data

In [ ]:
feat_gat_tr = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/gender_age_train.csv')
feat_gat_test = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/gender_age_test.csv')
feat_ph = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/phone_brand_device_model.csv')
# removing duplicate values in phone dataframe so doesn't create problems while joining dataframes
feat_ph=feat_ph.drop_duplicates('device_id',keep='first')

feat_eve = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/events.csv'parse_dates=['timestamp'],infer_datetime_format=True)

feat_ap_eve = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/app_events.csv',dtype={'is_installed':bool, 'is_active':bool})

feat_app_lab = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/app_labels.csv') 
feat_fold_5=pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/folds_5.csv') 
feat_fold_10=pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/folds_10.csv')

In [ ]:
import scipy 
Xtrain=scipy.sparse.load_npz('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/Xtrain_all.npz')  
Xtest=scipy.sparse.load_npz('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/Xtest_all.npz')

In [ ]:
ytrain = feat_gat_tr['group']
label_group = LabelEncoder()
ytrain = label_group.fit_transform(ytrain)

## Hyperparameter tuning on Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
param_grid = [
    {'classifier' : [DecisionTreeClassifier()],
    'classifier__min_samples_split' : list(range(2,10,2)),
    'classifier__max_depth' : list(range(6,22,5))}
]
feat_clf = GridSearchCV(pipe, param_grid = param_grid, scoring='neg_log_loss', cv = 5, verbose=True, n_jobs=-1) 
best_clf=feat_clf.fit(Xtrain,ytrain)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  5.2min finished


In [ ]:
best_clf.best_params_

{'classifier': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=6,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=None, splitter='best'),
 'classifier__max_depth': 6,
 'classifier__min_samples_split': 6}

In [ ]:
best_clf.best_score_

-2.4332835621797053

## Predicting on best model Dtree

In [ ]:
from sklearn.linear_model import LogisticRegression 
feat_lr=DecisionTreeClassifier(max_depth= 6,min_samples_split= 6).fit(Xtrain,ytrain) 

p_group=np.zeros((Xtest.shape[0],12)) 
p_group=feat_lr.predict_proba(Xtest)

In [ ]:
pred_test=pd.DataFrame(p_group,index=feat_gat_tr.device_id,columns=list(lab.classes_)) 
pred_test=pred_test.reset_index() 

pred_test.to_csv('pred_test_dt.csv',index=False)